not done.

the model can't predict never seen data.

In [120]:
import numpy as np
from scipy.stats import skew

from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
import seaborn as sns

import pandas as pd

In [11]:
# expected = 0
print(skew(a= [[-1],
               [0],
               [1]],
           axis=0))

[0.]


In [91]:
#              X1    y
X_train = np.array([
              [0], # 0
              [0], # 0
              [0], # 0
              [1], # 1
              [1], # 1
              [1], # 1
              [2], # 0
              [2], # 0
              [2], # 0
              [3], # 1
              [3], # 1
              [3], # 1
              [4], # 0
              [4], # 0
              [4], # 0
              [5], # 1
              [5], # 1
              [5], # 1
              [6], # 0
              [6], # 0
              [6], # 0
              [7], # 1
              [7], # 1
              [7], # 1
             ]
            )

# skew is 0, can use MinMaxScaler
# print(skew(a= X,
#            axis=0))

scaler = MinMaxScaler().fit(X=X_train)

X_train_scaled = scaler.transform(X=X_train)

#             X1
y_train = [
     0,0,0, # 0
     1,1,1, # 1
     0,0,0, # 2
     1,1,1, # 3
     0,0,0, # 4
     1,1,1, # 5
     0,0,0, # 6
     1,1,1, # 7
    ]

clf = LogisticRegression()
clf.fit(X= X_train_scaled, y= y_train)

# seen data
# X_test = np.array([[4],
#                    [4],
#                    [4],
#                    [5],
#                    [5],
#                    [5]])

# never seen data
X_test = np.array([[8],
                   [8],
                   [8],
                   [9],
                   [9],
                   [9]])

y_test = np.array([0,
                   0,
                   0,
                   1,
                   1,
                   1])

X_test_scaled = scaler.transform(X=X_test)

y_test_pred = clf.predict(X=X_test_scaled)

print("y_true", y_test)
print("y_pred", y_test_pred)

# average=None is important.
# otherwise you get average='binary', pos_label=1
print("accuracy score", accuracy_score(y_true=y_test, y_pred=y_test_pred))
print("recall score", recall_score(y_true=y_test, y_pred=y_test_pred))
print("precision score", precision_score(y_true=y_test, y_pred=y_test_pred, average=None, zero_division=0))
print("f1 score", f1_score(y_true=y_test, y_pred=y_test_pred, average=None))


y_true [0 0 0 1 1 1]
y_pred [1 1 1 1 1 1]
accuracy score 0.5
recall score 1.0
precision score [0.  0.5]
f1 score [0.         0.66666667]


In [184]:
def cross_validate_display(estimator: dict):
    scoring = {'recall': make_scorer(score_func=recall_score, pos_label=1),
               'precision': make_scorer(score_func=precision_score, pos_label=1, zero_division=0),
               'f1': make_scorer(score_func=f1_score, pos_label=1),
               'accuracy_score': make_scorer(score_func=accuracy_score)}
            
    cv_result = cross_validate(estimator=estimator, X=X_train_scaled, y=y_train,
                               scoring=scoring, return_train_score=True)

    # Arithmetic mean because the number of samples are equal.
    get_mean_std = lambda array: [array.mean(), array.std()]

    with pd.option_context("display.float_format", "{:.2f}".format):
        display(pd.DataFrame(data={
            'train recall': get_mean_std(cv_result['train_recall']),
            'test recall': get_mean_std(cv_result['test_recall']),
            'train precision': get_mean_std(cv_result['train_recall']),
            'test precision': get_mean_std(cv_result['test_recall']),
            'train f1': get_mean_std(cv_result['train_f1']),
            'test f1': get_mean_std(cv_result['test_f1']),
            'train accuracy_score': get_mean_std(cv_result['train_accuracy_score']),
            'test accuracy_score': get_mean_std(cv_result['test_accuracy_score']),
                             },
                             index=['mean', 'std']).T)
    
cross_validate_display(estimator=clf)

,mean,std
train recall,0.66,0.20
test recall,0.57,0.39
train precision,0.66,0.20
test precision,0.57,0.39
train f1,0.60,0.09
test f1,0.54,0.29
train accuracy_score,0.58,0.06
test accuracy_score,0.66,0.12


In [181]:
def GridSearchCV_display(cv_results: dict):
    cv_results_df = pd.DataFrame(cv_results)
    cv_results_df['diff_mean_recall'] = cv_results_df['mean_test_recall'] - cv_results_df['mean_train_recall']
    cv_results_df['diff_mean_precision'] = cv_results_df['mean_test_precision'] - cv_results_df['mean_train_precision']
    cv_results_df['diff_mean_f1'] = cv_results_df['mean_test_f1'] - cv_results_df['mean_train_f1']
    cv_results_df['diff_mean_accuracy_score'] = cv_results_df['mean_test_accuracy_score'] - cv_results_df['mean_train_accuracy_score']

    with pd.option_context('display.max_colwidth', None, "display.float_format", "{:.2f}".format):
        display(cv_results_df[['params', 
                               'mean_test_recall', 'diff_mean_recall',
                               'mean_test_precision', 'diff_mean_precision',
                               'mean_test_f1', 'diff_mean_f1',
                               'mean_test_accuracy_score', 'diff_mean_accuracy_score']].sort_values(by=['mean_test_recall', 'diff_mean_recall'], ascending=False).T)

In [178]:
clf_cv = GridSearchCV(estimator=clf, 
                      scoring=scoring,
                      refit="f1",
                      param_grid={
                        'C': [0.1, 0.2, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                        'penalty': ['l1', 'l2'],
                        'solver': ['liblinear']
                      },
                      return_train_score=True)
clf_cv.fit(X=X_train_scaled, y=y_train)

print("best_params", clf_cv.best_params_)
print("best_score", clf_cv.best_score_)

GridSearchCV_display(clf_cv.cv_results_)

best_params {'C': 3, 'penalty': 'l1', 'solver': 'liblinear'}
best_score 0.6857142857142857


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


,8,10,12,3,1,6,13,15,20,5,...,16,18,17,19,21,22,23,0,2,4
params,"{'C': 3, 'penalty': 'l1', 'solver': 'liblinear'}","{'C': 4, 'penalty': 'l1', 'solver': 'liblinear'}","{'C': 5, 'penalty': 'l1', 'solver': 'liblinear'}","{'C': 0.2, 'penalty': 'l2', 'solver': 'liblinear'}","{'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}","{'C': 2, 'penalty': 'l1', 'solver': 'liblinear'}","{'C': 5, 'penalty': 'l2', 'solver': 'liblinear'}","{'C': 6, 'penalty': 'l2', 'solver': 'liblinear'}","{'C': 9, 'penalty': 'l1', 'solver': 'liblinear'}","{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}",...,"{'C': 7, 'penalty': 'l1', 'solver': 'liblinear'}","{'C': 8, 'penalty': 'l1', 'solver': 'liblinear'}","{'C': 7, 'penalty': 'l2', 'solver': 'liblinear'}","{'C': 8, 'penalty': 'l2', 'solver': 'liblinear'}","{'C': 9, 'penalty': 'l2', 'solver': 'liblinear'}","{'C': 10, 'penalty': 'l1', 'solver': 'liblinear'}","{'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}","{'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'}","{'C': 0.2, 'penalty': 'l1', 'solver': 'liblinear'}","{'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}"
mean_test_recall,0.87,0.77,0.77,0.73,0.67,0.67,0.57,0.57,0.57,0.57,...,0.57,0.57,0.47,0.47,0.47,0.47,0.47,0.00,0.00,0.00
diff_mean_recall,0.15,0.09,0.09,0.04,-0.04,-0.04,-0.03,-0.03,-0.03,-0.09,...,-0.09,-0.09,-0.09,-0.09,-0.09,-0.09,-0.09,0.00,0.00,0.00
mean_test_precision,0.60,0.69,0.69,0.66,0.46,0.45,0.49,0.49,0.49,0.50,...,0.49,0.49,0.29,0.29,0.29,0.29,0.29,0.00,0.00,0.00
diff_mean_precision,0.01,0.14,0.14,0.02,0.01,-0.01,-0.05,-0.05,-0.05,-0.09,...,-0.07,-0.07,-0.24,-0.24,-0.24,-0.24,-0.24,0.00,0.00,0.00
mean_test_f1,0.69,0.68,0.68,0.56,0.46,0.51,0.48,0.48,0.48,0.47,...,0.48,0.48,0.35,0.35,0.35,0.35,0.35,0.00,0.00,0.00
diff_mean_f1,0.06,0.09,0.09,-0.02,-0.08,-0.04,-0.08,-0.08,-0.08,-0.13,...,-0.11,-0.11,-0.19,-0.19,-0.19,-0.19,-0.19,0.00,0.00,0.00
mean_test_accuracy_score,0.58,0.62,0.62,0.50,0.46,0.50,0.54,0.54,0.54,0.54,...,0.54,0.54,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50
diff_mean_accuracy_score,-0.02,0.06,0.06,-0.06,-0.11,-0.09,-0.01,-0.01,-0.01,-0.04,...,-0.04,-0.04,-0.03,-0.03,-0.03,-0.03,-0.03,0.00,0.00,0.00


In [176]:
cross_validate_display(estimator=LogisticRegression(C=3, penalty='l1', solver='liblinear'))

{'fit_time': array([0., 0., 0., 0., 0.]),
 'score_time': array([0., 0., 0., 0., 0.]),
 'test_recall': array([1.  , 1.  , 0.33, 1.  , 1.  ]),
 'train_recall': array([1.  , 1.  , 0.56, 0.33, 0.7 ]),
 'test_precision': array([1.  , 0.4 , 0.33, 0.75, 0.5 ]),
 'train_precision': array([0.53, 0.62, 0.56, 0.6 , 0.64]),
 'test_f1': array([1.  , 0.57, 0.33, 0.86, 0.67]),
 'train_f1': array([0.69, 0.77, 0.56, 0.43, 0.67]),
 'test_accuracy_score': array([1. , 0.4, 0.2, 0.8, 0.5]),
 'train_accuracy_score': array([0.53, 0.68, 0.58, 0.58, 0.65])}

,mean,std
train recall,0.72,0.26
test recall,0.87,0.27
train precision,0.72,0.26
test precision,0.87,0.27
train f1,0.62,0.12
test f1,0.69,0.23
train accuracy_score,0.60,0.06
test accuracy_score,0.58,0.29


# concate X and y

In [8]:
def isolate():
    X = np.array([[1,2],
                  [3,4]])
    y = np.array([5,6])
    df = np.concatenate([X,y.reshape(-1,1)], axis=1)
    display(df)

isolate()

array([[1, 2, 5],
       [3, 4, 6]])